In [1]:
from datasets import load_dataset
import os

# Suppose your CSV is named 'checkworthy_data.csv' with columns: "text", "label"
train_path = os.path.join(os.getcwd(), "datasets", "cleaned_data_train.csv")
test_path = os.path.join(os.getcwd(), "datasets", "cleaned_data_test.csv")
data_files = {"train": train_path, "test": test_path}  # or separate train/test if you have them
raw_datasets = load_dataset("csv", data_files=data_files)


c:\Computer Science\Hackathons\Techfest 2025\techfest-2025\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [4]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

KeyboardInterrupt: 

In [14]:
import pandas as pd

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [15]:
train["label"].value_counts(normalize=True)

label
0    0.657875
1    0.342125
Name: proportion, dtype: float64

In [16]:
test["label"].value_counts(normalize=True)

label
0    0.657885
1    0.342115
Name: proportion, dtype: float64